In [1]:
import napari
import os
from tqdm import tqdm
import numpy as np
import glob2 as glob
import zarr
import json
import skimage.io as io
import pandas as pd
import umap.umap_ as umap
from sklearn.preprocessing import StandardScaler

# # set parameters
root = "E:\\Nick\\Cole Trapnell's Lab Dropbox\\Nick Lammers\\Nick\\killi_tracker\\"
project_name = "240219_LCP1_93hpf_to_127hpf"
image_dir = os.path.join(root, "built_data", project_name, "")
config_name = "tracking_cell.txt"# 
tracking_folder = config_name.replace(".txt", "")
tracking_folder = tracking_folder.replace(".toml", "")

tracking_directory = os.path.join(root, "built_data", "tracking", project_name, tracking_folder)
snip_directory = os.path.join(root, "built_data", "shape_snips", project_name, tracking_folder, "class0")

# load metadata
metadata_file_path = os.path.join(root, "metadata", project_name, "metadata.json")
f = open(metadata_file_path)
metadata = json.load(f)
scale_vec = np.asarray([metadata["ProbPhysicalSizeZ"], metadata["ProbPhysicalSizeY"], metadata["ProbPhysicalSizeX"]])

## Load tracking DF

In [2]:
# load tracking data
tracks_df = pd.read_csv(os.path.join(tracking_directory, "tracks.csv"))
tracks_df.head()

,track_id,t,z,y,x,id,parent_track_id,parent_id
0,1,0,2.0,117.0,57.0,1000001,-1,-1
1,1,1,2.0,118.0,56.0,2000001,-1,1000001
2,1,2,2.0,119.0,55.0,3000001,-1,2000001
3,1,3,2.0,119.0,55.0,4000001,-1,3000001
4,1,4,2.0,120.0,57.0,5000001,-1,4000001


## Load sphere coordinate DF

In [3]:
# load tracking data
sphere_df = pd.read_csv(os.path.join(root, "metadata", project_name, "sphere_df.csv"))
sphere_df.tail()
si = np.argsort(sphere_df["t"])
sphere_df = sphere_df.loc[si, :]
sphere_df.reset_index(inplace=True)

In [4]:
# generate smoothed sphere coordinates
window_size = 5
sphere_df["ZM"] = sphere_df["Z"].rolling(window_size, center=True, min_periods=1).mean() #/ scale_vec[0]
sphere_df["YM"] = sphere_df["Y"].rolling(window_size, center=True, min_periods=1).mean() #/ scale_vec[1]
sphere_df["XM"] = sphere_df["X"].rolling(window_size, center=True, min_periods=1).mean() #/ scale_vec[2]
sphere_df["rm"] = sphere_df["r"].rolling(window_size, center=True, min_periods=1).mean() #/ scale_vec[0]

tracks_df["zum"] = tracks_df["z"] *scale_vec[0]
tracks_df["yum"] = tracks_df["y"] *scale_vec[1]
tracks_df["xum"] = tracks_df["x"] *scale_vec[2]
sphere_df.head()

,index,Z,Y,X,r,t,project,ZM,YM,XM,rm
0,322,-130.922350,612.522760,638.747552,585.680548,0,240219_LCP1_93hpf_to_127hpf,-131.840868,612.039732,640.845995,586.166719
1,323,-123.378080,605.774383,642.860044,582.202359,1,240219_LCP1_93hpf_to_127hpf,-134.918153,615.005009,640.965030,587.720373
2,324,-141.222175,617.822052,640.930389,590.617249,2,240219_LCP1_93hpf_to_127hpf,-136.435576,617.009057,641.109812,588.214346
3,325,-144.150005,623.900840,641.322136,592.381337,3,240219_LCP1_93hpf_to_127hpf,-139.141170,620.080003,641.838164,589.142819
4,326,-142.505268,625.025248,641.688937,590.190236,4,240219_LCP1_93hpf_to_127hpf,-144.301180,624.557621,641.668935,591.334294


## 3D visuals

In [5]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

def ms(x, y, z, radius, resolution=20):
    """Return the coordinates for plotting a sphere centered at (x,y,z)"""
    u, v = np.mgrid[0:2*np.pi:resolution*2j, 0:np.pi:resolution*1j]
    X = radius * np.cos(u)*np.sin(v) + x
    Y = radius * np.sin(u)*np.sin(v) + y
    Z = radius * np.cos(v) + z
    return (X, Y, Z)

In [7]:
plotly_repmap = np.asarray(px.colors.qualitative.Plotly*100)
tracks_df["plot_color"] = plotly_repmap[tracks_df["track_id"]]

In [8]:
from tqdm import tqdm

time_id = 700
mem = 10
time_index = np.unique(tracks_df["t"])
figure_root = os.path.join(root, "figures", project_name, tracking_folder)
if not os.path.isdir(figure_root):
    os.makedirs(figure_root)
    
track_path_3d = os.path.join(figure_root, "track_plots_3d")
if not os.path.isdir(track_path_3d):
    os.makedirs(track_path_3d)

for time_id in tqdm(time_index):
    tracks_df_ft = tracks_df.loc[tracks_df["t"]==time_id]
    tracks_df_ft_mem = tracks_df.loc[(tracks_df["t"]<=time_id) & (tracks_df["t"]>time_id-mem) ]
    x_pns_surface, y_pns_surface, z_pns_suraface = ms(sphere_df.loc[time_id, "XM"], 
                                                      sphere_df.loc[time_id, "YM"], 
                                                      sphere_df.loc[time_id, "ZM"],sphere_df.loc[time_id, "rm"]*0.95)

    point_array = tracks_df_ft.loc[:, ["xum", "yum", "zum"]].to_numpy()
    point_ids = tracks_df_ft.loc[:, ["track_id"]].to_numpy().flatten()
    point_colors = tracks_df_ft.loc[:, ["plot_color"]].to_numpy().flatten()
    track_array = tracks_df_ft_mem.loc[:, ["xum", "yum", "zum"]].to_numpy()
    track_ids = tracks_df_ft_mem.loc[:, ["track_id"]].to_numpy().flatten()
    track_colors = tracks_df_ft_mem.loc[:, ["plot_color"]].to_numpy().flatten()

    extant_ids = np.unique(track_ids)

    fig = go.Figure()
    fig.add_trace(go.Surface(x=x_pns_surface, y=y_pns_surface, z=z_pns_suraface, opacity=0.75, 
                             colorscale="Blues"))#, lighting=dict(ambient=0.5)))

    for t in range(len(extant_ids)-1):
        tid = extant_ids[t]
        point_ft = point_ids==tid
        track_ft = track_ids==tid

        if np.sum(point_ft) > 0:
            fig.add_trace(go.Scatter3d(x=point_array[point_ft, 0], y=point_array[point_ft, 1], z=point_array[point_ft, 2], 
                                       mode="markers", showlegend=False, marker=dict(color=point_colors[point_ft], size=6)))

        fig.add_trace(go.Scatter3d(x=track_array[track_ft, 0], y=track_array[track_ft, 1], z=track_array[track_ft, 2], 
                                   mode="lines", showlegend=False, line=dict(color=track_colors[track_ft])))

    fig.update_layout(scene = dict(
                        xaxis_title='x (um)',
                        yaxis_title='y (um)',
                        zaxis_title='z (um)'))

    camera = dict(
        up=dict(x=0, y=0, z=1),
        center=dict(x=0, y=0, z=0),
        eye=dict(x=1, y=1, z=2)
    )

    fig.update_layout(scene_camera=camera)
    
    fig.write_image(os.path.join(track_path_3d, f'frame3d_t{time_id:04}.png'), scale=3)

fig.show()

  0%|          | 0/1350 [00:00<?, ?it/s]


ValueError: 
Image export using the "kaleido" engine requires the kaleido package,
which can be installed using pip:
    $ pip install -U kaleido


## 2D visuals

In [ ]:
from astropy.coordinates import cartesian_to_spherical

time_id = 700

tracks_df_ft = tracks_df.loc[tracks_df["t"]==time_id]
track_coords_cart = tracks_df_ft.loc[:, ["x", "y", "z"]].to_numpy()
sphere_center = np.asarray([sphere_df.loc[time_id, "XM"], sphere_df.loc[time_id, "YM"], sphere_df.loc[time_id, "ZM"]])
track_coords_centered = track_coords_cart - sphere_center

track_coords_sph = cartesian_to_spherical(track_coords_centered[:, 0], track_coords_centered[:, 1], track_coords_centered[:, 2])
track_array = np.asarray([c.value for c in track_coords_sph]).T


fig = px.scatter(x=track_array[:, 2], y=track_array[:, 1])
fig.show()

In [ ]:
from tqdm import tqdm
dt = 90

window_size_vec = [5, 25]

for window_size in window_size_vec:
    w_str = str(window_size)
    c_time = dt*(window_size-1)
    buffer_size = window_size // 2
    track_index = np.unique(tracks_df["track_id"])

    for t, track_id in enumerate(tqdm(track_index)):
        track_bool = tracks_df["track_id"] == track_id

        if np.sum(track_bool) >= window_size:
            xyz = tracks_df.loc[track_bool, ["x", "y", "z"]].to_numpy()

            # get contour length
            d1 = np.sqrt(np.sum(np.diff(xyz, 1, axis=0)**2, axis=1))
            d1_cs = np.cumsum(d1)
            cs1 = d1_cs[window_size-2:]
            cs2 = np.concatenate(([0], d1_cs[:-window_size+1]))
            contour_r = cs1 - cs2

            # get net displacement
            net_r = np.sqrt(np.sum((xyz[window_size-1:, :] - xyz[:-window_size+1, :])**2, axis=1))

            # assign to tracks df
            to_indices = np.where(track_bool)[0]
            to_indices = to_indices[buffer_size:-buffer_size]
            tracks_df.loc[to_indices, "speed_" + w_str] = contour_r / c_time
            tracks_df.loc[to_indices, "speed_net_" + w_str] = net_r / c_time
            tracks_df.loc[to_indices, "coherence_" + w_str] = net_r / contour_r


        

In [ ]:
import plotly.express as px

track_id = 101
fig = px.scatter(tracks_df.loc[tracks_df["track_id"]==track_id], x="t", y="speed_5", color="t", template="plotly")
fig.show()

In [ ]:
fig = px.density_contour(tracks_df, x="speed_25", y="speed_net_25")
fig.update_traces(contours_coloring="fill", contours_showlabels = True, colorscale="Blues")
fig.show()

In [ ]:
fig = px.density_contour(tracks_df, x="t", y="coherence_25")
fig.update_traces(contours_coloring="fill", contours_showlabels = True, colorscale="Blues")
fig.show()

In [ ]:
shape_df = pd.read_csv(os.path.join(tracking_directory, "cell_shape_df.csv"))
shape_df.head()

In [ ]:
fig = px.density_contour(shape_df, x="t", y="circularity")
fig.update_traces(contours_coloring="fill", contours_showlabels = True, colorscale="Blues")
fig.show()

In [ ]:
master_df = tracks_df.merge(shape_df, on=["track_id", "t"], how="left")

In [ ]:
master_df["time_id"] = np.floor(master_df["t"].to_numpy()/350).astype(int)

fig = px.density_contour(master_df, x="complexity", y="speed_25", color="time_id", template="plotly")
# fig.update_traces(contours_coloring="fill", contours_showlabels = True, colorscale="Blues")
fig.show()

In [ ]:
fig = px.density_contour(master_df, x="circularity", y="speed_25")
fig.update_traces(contours_coloring="fill", contours_showlabels = True, colorscale="Blues")
fig.show()